In [40]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import pylab as plt
import sys
sys.path.append(r'/nfs/iil/disks/hip_ana_sim_01/dgottesm/analysis_and_tools/jupyter_notebooks/brk_scope/PostProcess/global_functions/')
from generate_data import rc_on_data
from ipywidgets import interact
import ipywidgets as widgets

In [51]:
samples=2000
# we want max min values to be +-0.2v
raw_data_pam2=np.random.randint(0, 2, samples).astype(float)/2.5-0.2
raw_data_pam4=np.random.randint(0, 4, samples).astype(float)/3*0.4-0.2
oversample = 20
cdr_oversample_options=list(range(20))
tau_in_ui=widgets.FloatSlider(value=0.3, min=0.1, max=1.4, step=0.1, description='tau_in_ui:', 
                                 continuous_update=False, orientation='horizontal', 
                                 readout_format='.1f', layout=widgets.Layout(width='50%'), style=dict(description_width='initial'))
noise_std_mv=widgets.FloatSlider(value=5, min=0, max=20, step=1, description='noise_std_mv:', 
                                 continuous_update=False, orientation='horizontal', 
                                 readout_format='.1f', layout=widgets.Layout(width='50%'), style=dict(description_width='initial'))
cdr_offset=widgets.FloatSlider(value=0, min=0, max=100-100/oversample, step=100/oversample, description='cdr offset % ui:', 
                                 continuous_update=False, orientation='horizontal', 
                                 readout_format='.1f', layout=widgets.Layout(width='50%'), style=dict(description_width='initial'))


@interact(tau_in_ui=tau_in_ui, noise_std_mv=noise_std_mv, cdr_offset=cdr_offset)
def plotme(tau_in_ui, noise_std_mv, cdr_offset, pam4=False):
    global df
    fig, ax = plt.subplots(2, 1, figsize=(15, 5))
    raw_data = (raw_data_pam4 if pam4 else raw_data_pam2) + np.random.normal(0,noise_std_mv*1e-3,samples)
    samples_in_window = 100
    
    plot = False
    oversample_that_cdr_focus_on=cdr_offset//(100/oversample)  # we want to cast the 0% to 100% of the cdr to oversample to focus on
    df = rc_on_data(raw_data=raw_data, oversample=oversample, 
                    samples_in_window=samples_in_window, tau_in_samples_unit_time=tau_in_ui, plot=plot)
    df['cdr_cursor']=df.query(f'oversampling=={oversample_that_cdr_focus_on}').assign(cdr_cursor=lambda r:r.outputs).cdr_cursor
    df.cdr_cursor.dropna().plot.hist(bins=200, ax=ax[0], title='hist of raw data', grid=True)
    df[['inputs','outputs','cdr_cursor']].tail(800).plot(style=['-.','-.','^'], ax=ax[1], title='raw data', grid=True)

interactive(children=(FloatSlider(value=0.3, continuous_update=False, description='tau_in_ui:', layout=Layout(…